## Access an SPSS ML model
This notebook shows how to access the information to a model withn a Watson Machine learning service

The first step is to get the credentials from the Watson service. This can be found in the user's bluemix account.

The first thing to do is to put the proper `wml_credentials` in the first cell.

Note that the notebook assumes a model name of `SPSS Churn Model` with only one deployment.<br/>
You can change that name by modifying the `model_name` variable in the first cell.

In [ ]:
import urllib3, requests, json

# retrieve your wml_service_credentials_username, wml_service_credentials_password, and wml_service_credentials_url from the
# Service credentials associated with your IBM Cloud Watson Machine Learning Service instance

wml_credentials= {
  "apikey": "X. . .kooOV",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/e46675b7f1bf89b09b5badfb3bd4a7b5:672f8d2a-4f11-41bf-96ce-a92cccf5cfa8::",
  "iam_apikey_name": "auto-generated-apikey-ab3b8bf1-87ed-4242-a6ab-0cd3aa3cd23f",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/e46675b7f1bf89b09b5badfb3bd4a7b5::serviceid:ServiceId-ededa4b9-4515-418c-92cf-cd84e98f8bac",
  "instance_id": "672f8d2a-4f11-41bf-96ce-a92cccf5cfa8",
  "password": "d7b. . .444a6682018",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "ab3. . .aa3cd23f"
}

# Get an IAM token from IBM Cloud
url     = "https://iam.bluemix.net/oidc/token"
headers = { "Content-Type" : "application/x-www-form-urlencoded" }
data    = "apikey=" + wml_credentials["apikey"] + "&grant_type=urn:ibm:params:oauth:grant-type:apikey"
IBM_cloud_IAM_uid = "bx"
IBM_cloud_IAM_pwd = "bx"
response  = requests.post( url, headers=headers, data=data, auth=( IBM_cloud_IAM_uid, IBM_cloud_IAM_pwd ) )
iam_token = response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + iam_token}
model_name = 'SPSS Churn Model'

## Explore the Watson Machine Learning Service
The next several cells shows us how to get to the information related to the models attached to this service and their publishing and deployment information. 

In [ ]:
# Get the information about the Watson service instance. 
response_instances = requests.get(wml_credentials['url'] + "/v3/wml_instances/" + wml_credentials['instance_id'], headers=header)
instances_json = json.loads(response_instances.text)
print (json.dumps(instances_json, indent=2, sort_keys=True))

In [ ]:
# Get the information on the published models
response_published = requests.get(instances_json['entity']['published_models']['url'], headers=header)
published_json = json.loads(response_published.text)
print (json.dumps(published_json, indent=2, sort_keys=True))

## Select the information for a specific model
There could be multiple models in this service, we find the one that corresponds to the SPSS model

In [ ]:
# Find the entry for the proper servide, display the index and name to make sure we got the right one
model_index=-1
# print(published_json['resources'][0]['entity']['name'])
for ix in range(len(published_json['resources'])) :
    if (published_json['resources'][ix]['entity']['name'] == model_name) :
        model_index = ix
print('Index ' + str(model_index) + ' - ' + published_json['resources'][model_index]['entity']['name'])

In [ ]:
# Display the fields required for the scoring and their types
for ix in range(len(published_json['resources'][model_index]['entity']['input_data_schema']['fields'])) :
    print(published_json['resources'][model_index]['entity']['input_data_schema']['fields'][ix]['name'] + ',' +
          published_json['resources'][model_index]['entity']['input_data_schema']['fields'][ix]['type'])

In [ ]:
# Get the information on the deployed models
response_deployed = requests.get(published_json['resources'][model_index]['entity']['deployments']['url'], headers=header)
deployed_json = json.loads(response_deployed.text)
print (json.dumps(deployed_json, indent=2, sort_keys=True))

## Prepare some data and then score it

In [ ]:
# Initialize test data
# Customer 1: "T", Customer 6: "F"
cust_1 = [1, "?", "F","S",1.000000,38000.000000,"N",24.393333,23.560000,0.000000,206.080000,0.000000,"CC","Budget","Intnl_discount",229.640000,3.000000]
cust_6 = [6, "?", "M","M",2.000000,29616.000000,"N",49.426667,29.780000,0.000000,45.500000,0.000000,"CH","FreeLocal","Standard",75.290000,2.000000]

In [ ]:
# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload_scoring = {"fields": ["ID", "CHURN", "Gender", "Status", "Children", "Est Income", "Car Owner", "Age", "LongDistance", "International", "Local", "Dropped", "Paymethod", "LocalBilltype", "LongDistanceBilltype", "Usage", "RatePlan"], "values": [cust_1, cust_6]}
response_scoring = requests.post(deployed_json['resources'][0]['entity']['scoring_url'], json=payload_scoring, headers=header)

print("Scoring response code:" + str(response_scoring))
scoring_json = json.loads(response_scoring.text)
print(json.dumps(scoring_json, indent=2, sort_keys=True) )

In [ ]:
# Print ID, $XF-CHURN, $XFC-CHURN
for ix in range(len(scoring_json['values'])) :
    print("ID: " + str(scoring_json['values'][ix][0]) + ', result: ' + str(scoring_json['values'][ix][17:]))